In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [27]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [28]:
def clean_reviews(no_reviews,dataset):
    import re
    import nltk
    nltk.download('stopwords') 
    from nltk.corpus import stopwords 
    
    my_stopwords = stopwords.words('english') 
            
    from nltk.stem.porter import PorterStemmer
     
    corpus =[] 
    if no_reviews == 35000:
        start = 0
        end = 35000
    else:
        start = 35000
        end = 50000
    for i in range (start, end):
        review = re.sub('[^a-zA-Z]',' ', dataset[i]) #Removes all other characters besides the letters and puts the review in a string
        review = review.lower() 
        review = review.split()
        ps = PorterStemmer() 
        review = [ps.stem(word) for word in review if not word in set (my_stopwords)]
        review = ' '.join(review) 
        corpus.append(review)
    return corpus

In [29]:
train = df.loc[0:34999]
test = df.loc[35000:]
train_data = train['review']
test_data = test['review']
train_labels = train['label']
test_labels = test['label']

In [30]:
train_data = clean_reviews(35000,train_data)
test_data = clean_reviews(50000,test_data)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
train_labels = train_labels.map({'pos': 1, 'neg': 0})
test_labels = test_labels.map({'pos': 1, 'neg': 0})

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
train_data = vect.fit_transform(train_data)

In [33]:
test_data = vect.transform(test_data)

In [34]:
from sklearn.tree import DecisionTreeClassifier
nb = DecisionTreeClassifier()

In [35]:
nb.fit(train_data, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [36]:
y_pred_class = nb.predict(test_data)

In [37]:
from sklearn import metrics
metrics.accuracy_score(test_labels, y_pred_class)

0.7174666666666667

In [38]:
metrics.confusion_matrix(test_labels, y_pred_class)

array([[5370, 2035],
       [2203, 5392]], dtype=int64)